In [ ]:
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
num_classes = 1
batch_size = 1
sample_size = 1
sequence_length = 1
learning_rate = 0.01
hidden_layer_size = 10


def norm_array(array):
    mean = np.mean(array)
    stddev = np.sqrt(np.sum(np.square(array - mean)) / len(array - 1))
    normalized_array = ((array - mean) / stddev)
    return normalized_array, mean.reshape(batch_size), stddev.reshape(batch_size)

def norm_test(array, mean, stddev):
    return (array - mean) / stddev

def test_decoder(array, mean, stddev):
    return (array * stddev) + mean

def get_linear_layer(vector):
    return(tf.matmul(vector, Wl) + bl)
def train_test_split(X, y, train_size):
    X_train = X[:int(len(X) // (1 / train_size))]
    y_train = y[:int(len(y) // (1 / train_size))]
    X_test = X[int(len(X) // (1 / train_size)):]
    y_test = y[int(len(y) // (1 / train_size)):]
    return X_train, X_test, y_train, y_test

def data_labels(data, sample_size, batch_size):
    labels = data.iloc[:sample_size, 2:]
    data = data.iloc[:sample_size, 1:]
    indeces = np.random.choice(len(data), size=batch_size, replace=False)
    batch_x = np.array(data.loc[indeces[0]][data.loc[indeces[0]][:] > 0][:-1])
    batch_y = np.array(labels.loc[indeces[0]][labels.loc[indeces[0]][:] > 0])
    return (batch_x, batch_y)


def cell_list(num_LSTM_layers):
    cell_list = []
    for i in range(num_LSTM_layers):
        cell_list.append(tf.contrib.rnn.BasicLSTMCell(hidden_layer_size, forget_bias=1.0))
    return cell_list


In [ ]:
data_monthly = pd.read_csv(r"C:\M4datasets\Monthly-train.csv")
data_month, labels_month = data_labels(data_monthly, sample_size = 1, batch_size = 1)
data_month = data_month[:90]
labels_month = labels_month[:90]

In [ ]:
num_layers = [1]
num_hidden = np.arange(25, 250, 2)
param_list_2 = [(x, y) for x in num_layers for y in num_hidden]

# num_layers = [1, 2]
# num_hidden = np.arange(25, 250, 2)
# param_list_2 = [(x, y) for x in num_layers for y in num_hidden]

In [ ]:
stop_iter = 912
stop_iters = []
stop_params = []
for param in param_list_2:
    num_LSTM_layers = param[0]
    hidden_layer_size = param[1]
    tf.reset_default_graph()
    with tf.name_scope("Training_data"):
        inputs = tf.placeholder(tf.float32,shape=[batch_size, None, sequence_length], name='inputs')
        y = tf.placeholder(tf.float32, shape=[batch_size], name='inputs')

    with tf.name_scope('Linear_weights_and_biases'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean = 0, stddev = 0.01))
        bl = tf.Variable(tf.truncated_normal([num_classes], mean = 0, stddev = 0.1))

    with tf.name_scope('Means_and_standard_deviations'):
        mean = tf.placeholder(tf.float32, shape = [batch_size])
        stddev = tf.placeholder(tf.float32, shape = [batch_size])

    with tf.variable_scope('LSTM_cell'):  
        cell = tf.contrib.rnn.MultiRNNCell(cells=cell_list(num_LSTM_layers), state_is_tuple=True)
        outputs, states = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)


    with tf.variable_scope('Batch_outputs'):
        final_outputs = tf.placeholder

    linear_output = get_linear_layer(outputs[0])
    final_output = test_decoder(linear_output, mean, stddev)[-1]
    mse = tf.reduce_mean(tf.squared_difference(final_output, y[0]))
    train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mse)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    mse_list = []
    for i in range(2000):
        batch_x, batch_y = data_day, labels_day
        data_train, data_test, labels_train, labels_test = train_test_split(batch_x, batch_y, train_size = 0.8)

        normed_data_train, mean_batch, stddev_batch = norm_array(data_train)

        normed_data_train = normed_data_train.reshape(batch_size, len(data_train), sequence_length)

        label = labels_train[-1].reshape(batch_size)
        sess.run(train_step, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        mse_batch = sess.run(mse, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        for j in range(len(data_test)):
            data_train =  np.append(data_train, data_test[j])
            labels_train = np.append(labels_train, labels_test[j])
            normed_data_train, mean_batch, stddev_batch = norm_array(data_train)
            normed_data_train = normed_data_train.reshape(batch_size, len(data_train), sequence_length)

            label = labels_train[-1].reshape(batch_size)
            sess.run(train_step, feed_dict = {inputs : normed_data_train, y : label, mean : mean_batch, stddev : stddev_batch})
            mse_batch += sess.run(mse, feed_dict = {inputs : normed_data_train,\
                                                y : label,  mean : mean_batch, stddev : stddev_batch})
        mse_batch = mse_batch / len(data_test)
        if i % 100 == 0:
            print(i)
        if mse_batch <= 1 and i < stop_iter: #For daily data important to change mse_batch <= 0.01
            stop_iter = i
            stop_iters.append(stop_iter)
            stop_params.append(param)
            print(stop_iter, param)
            break
        if i >= stop_iter:
            break
    sess.close()
    print(param, stop_iter)

print(stop_iters)
print(stop_params)